In [11]:
import pickle
import pandas as pd
import re

# function to read data from file
def read_tagging(file_name,language):
    path="../data/"+language+"/tagging/"+file_name+".conllu"
    data = pd.read_csv (path, sep = '\t',quoting=3, names=["POSITION","WORD","TAG"])
    return data


def extract_sentences_from_dataframe(df):
    sentences = ''
    for index, row in df.iterrows():
        word = row['WORD']
        if pd.notnull(word):  # Se la parola non è nulla
            if sentences:  # Se c'è già una frase, aggiungi uno spazio prima della nuova parola
                sentences += ' '
            sentences += word
    return sentences

#deserialize data from a file
def load_data(file_name,language):
    path="../data/"+language+"/"+file_name
    try: 
        file = open(path, 'rb') 
        data = pickle.load(file) 
        return data
    except: 
        print("Error in reading data")



In [12]:
import pandas as pd

def extract_entities_from_dataframe(dataframe):
    entity_spans = []

    current_entity_span = None
    current_sentence_index = 0

    for index, row in dataframe.iterrows():
        word = row['WORD']
        tag = row['TAG']
        position = row['POSITION']

        if position == 0:  # Inizio di una nuova frase
            current_sentence_index += 1

        if tag != 'O':
            if tag.startswith('B-'):
                # Se inizia una nuova entità, chiudi quella corrente e inizia una nuova
                if current_entity_span is not None:
                    entity_spans.append(current_entity_span)
                current_entity_span = {'Tag': tag[2:], 'Sentence Number': current_sentence_index}
                current_entity_span['Start Index'] = index
                current_entity_span['End Index'] = index
            elif tag.startswith('I-'):
                # Aggiungi la parola all'entità corrente
                if current_entity_span is None:
                    print()
                else:
                    current_entity_span['End Index'] = index
            else:
                print("Errore: Tag non riconosciuto.")

        else:
            # Se il tag è "O" ma siamo all'interno di una serie di tag non "O", chiudi l'entità corrente
            if current_entity_span is not None:
                entity_spans.append(current_entity_span)
                current_entity_span = None

    # Aggiungi l'ultima entità se presente
    if current_entity_span is not None:
        entity_spans.append(current_entity_span)

    # Creazione del dataframe di output
    output_data = {'Tag': [], 'Sentence Number': [], 'Start Index': [], 'End Index': []}
    for entity_span in entity_spans:
        output_data['Tag'].append(entity_span['Tag'])
        output_data['Sentence Number'].append(entity_span['Sentence Number'])
        output_data['Start Index'].append(entity_span['Start Index'])
        output_data['End Index'].append(entity_span['End Index'])

    output_df = pd.DataFrame(output_data)

    return output_df

In [13]:
import pandas as pd

def calculate_accuracy(system_df, golden_df):
    # Uniamo i due dataframe per confrontare i tag
    merged_df = pd.merge(system_df, golden_df, left_index=True, right_index=True, suffixes=('_system', '_golden'))
    
    # Contiamo quante volte i tag corrispondenti sono uguali
    correct_tags = (merged_df['TAG_system'] == merged_df['TAG_golden']).sum()
    
    # Calcoliamo l'accuratezza
    accuracy = correct_tags / len(system_df)
    
    # Converti in percentuale e arrotonda alla prima cifra decimale
    accuracy_percent = round(accuracy * 100, 1)
    
    return accuracy_percent



In [14]:
import pandas as pd

def calculate_precision_recall(predicted_df, golden_df):
    # Unione delle entità predette e delle entità del sistema dorato
    merged_df = pd.merge(predicted_df, golden_df, how='outer', indicator=True)

    # Calcolo dei true positives (TP), false positives (FP) e false negatives (FN)
    TP = merged_df[(merged_df['_merge'] == 'both')].shape[0]
    FP = merged_df[(merged_df['_merge'] == 'right_only')].shape[0]
    FN = merged_df[(merged_df['_merge'] == 'left_only')].shape[0]

    # Calcolo della precisione e del recall
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    # Converti in percentuali e arrotonda alla prima cifra decimale
    precision_percent = round(precision * 100, 1)
    recall_percent = round(recall * 100, 1)

    return precision_percent, recall_percent



In [16]:
import pandas as pd
from datetime import datetime

def evaluate_and_save_results(languages, output_file, dataset_size, smoothing_type):

    # Scrivi i risultati su file
   
    with open(output_file, "a") as file:
        file.write("##################################\n")
        file.write(f"Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        file.write(f"Dimensione del dataset: {dataset_size}\n")
        file.write(f"Tipo di Smoothing: {'Smoothing Type ' + str(smoothing_type)}\n\n")
        file.write("Statistiche aggiuntive:\n")
        file.write("Language | Acc Viterbi | Precision Viterbi | Recall Viterbi | Acc Nayve | Precision Nayve | Recall Nayve | Acc Viterbi Sent | Precision Viterbi Sent | Recall Viterbi Sent\n")
        
        for language in languages:
            tot_vit_df = read_tagging("viterbi_tag", language)
            sent_vit_df= read_tagging("sent_viterbi_tag", language)
            nayve_df = read_tagging("nayve_tag", language)
            golden_df = read_tagging("golden_tag", language)
            
            gol_quadruples_df = extract_entities_from_dataframe(golden_df)
            nay_quadruples_df = extract_entities_from_dataframe(nayve_df)
            tot_vit_quadruples_df = extract_entities_from_dataframe(tot_vit_df)
            sent_vit_quadruples_df = extract_entities_from_dataframe(sent_vit_df)
            

            acc_vit = calculate_accuracy(tot_vit_df, golden_df)
            acc_vit_sent = calculate_accuracy(sent_vit_df, golden_df)
            acc_nayve = calculate_accuracy(nayve_df, golden_df)
            precision_v, recall_v = calculate_precision_recall(tot_vit_quadruples_df, gol_quadruples_df)
            precision_v_sent, recall_v_sent = calculate_precision_recall(sent_vit_quadruples_df, gol_quadruples_df)
            precision_n, recall_n = calculate_precision_recall(nay_quadruples_df, gol_quadruples_df)

            file.write(f"{language} || {format(acc_vit, '.1f')} % | {format(precision_v, '.1f')} % | {format(recall_v, '.1f')} % ||| {format(acc_nayve, '.1f')} % | {format(precision_n, '.1f')} % | {format(recall_n, '.1f')} % ||| {format(acc_vit_sent, '.1f')} % | {format(precision_v_sent, '.1f')} % | {format(recall_v_sent, '.1f')} % |\n")
        
        file.write("##################################\n \n")

# Esempio di utilizzo
languages = ["en", "it", "es"]
output_file = "../evaluation_results.txt"
dataset_size = 1000
smoothing_type = 4
evaluate_and_save_results(languages, output_file, dataset_size, smoothing_type)


KeyboardInterrupt: 